# Instalaciones

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from opacus import PrivacyEngine
import matplotlib.pyplot as plt

# Paso 1: Procesamiento de Datos

In [5]:
# Cargar y dividir el conjunto de datos
breast_dataset = load_breast_cancer()
X = breast_dataset.data
y = breast_dataset.target

# Normalizar los datos
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Convertir a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Paso 2: Definir el Modelo

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(30, 50)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, 1)
        # No necesitas una capa Sigmoid si usas BCEWithLogitsLoss

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Paso 3: Clase Cliente

In [93]:
class Client:
    def __init__(self, data, target, client_number):
        self.client_number = client_number
        self.model = Net()
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.01)
        self.criterion = nn.BCEWithLogitsLoss()
        dataset = TensorDataset(data, target.view(-1, 1))
        self.data_loader = DataLoader(dataset, batch_size=15, shuffle=True)
        self.privacy_engine = PrivacyEngine()  
        self.model, self.optimizer, self.data_loader = self.privacy_engine.make_private_with_epsilon(
            module=self.model,
            optimizer=self.optimizer,
            data_loader=self.data_loader,
            epochs=100,
            target_epsilon=100,
            target_delta=0.001,
            max_grad_norm=1,
        )
        
    def train(self, epochs=5):
        self.model.train()
        for epoch in range(epochs):
            total_correct = 0
            total_samples = 0
            for data, target in self.data_loader:
                self.optimizer.zero_grad()
                output = self.model(data)
                loss = self.criterion(output, target)
                loss.backward()
                self.optimizer.step()
                
                # Calcular el accuracy en los datos de entrenamiento
                pred = torch.sigmoid(output).round()
                correct = pred.eq(target.view_as(pred)).sum().item()
                total_correct += correct
                total_samples += target.size(0)
            
            train_accuracy = total_correct / total_samples  
            epsilon = self.privacy_engine.get_epsilon(delta=0.001)  
            
            # Imprimir la información, incluyendo el número del cliente
            print(f"Client: {self.client_number}, Epoch: {epoch+1}, Loss: {loss.item()}, Epsilon: {epsilon}, Train ACC: {train_accuracy * 100:.2f}%")

        return self.model.state_dict()
    
    # Método de evaluación  con la data de test para cada cliente
    def evaluate(self, test_data, test_target):
        self.model.eval()
        with torch.no_grad():
            output = self.model(test_data)
            pred = torch.sigmoid(output).round()
            correct = pred.eq(test_target.view_as(pred)).sum().item()
            accuracy = correct / len(test_target)
        return accuracy


# Paso 4: Clase Servidor

In [33]:

class Server:
    def __init__(self):
        self.global_model = Net()

    def aggregate(self, client_models):
        global_dict = {}
        for k in client_models[0].keys():
            adjusted_key = k.replace("_module.", "")  
            global_dict[adjusted_key] = torch.stack([client_models[i][k] for i in range(len(client_models))], 0).mean(0)
        self.global_model.load_state_dict(global_dict)

    def evaluate(self, test_data, test_target):
        self.global_model.eval()  
        with torch.no_grad():
            output_global = self.global_model(test_data)  # y aquí también
            pred_global = torch.sigmoid(output_global).round()
            correct_global = pred_global.eq(test_target.view_as(pred_global)).sum().item()
            accuracy_global = correct_global / len(test_target)
        return accuracy_global


# Paso 5: Entrenamiento y Evaluación

In [99]:
#Entrenamiento y evaluación con la división de prueba para cada cliente:

client_data = torch.chunk(X_train_tensor, 3)
client_target = torch.chunk(y_train_tensor, 3)


clients = [Client(client_data[i], client_target[i], i+1) for i in range(3)]
client_models = [client.train() for client in clients]
server = Server()
server.aggregate(client_models)

Client: 1, Epoch: 1, Loss: 0.618086040019989, Epsilon: 7.229711840478823, Train ACC: 93.14%
Client: 1, Epoch: 2, Loss: 0.5472254157066345, Epsilon: 9.618787125042179, Train ACC: 89.68%
Client: 1, Epoch: 3, Loss: 0.5537917613983154, Epsilon: 11.56035669629223, Train ACC: 94.51%
Client: 1, Epoch: 4, Loss: 0.4533534049987793, Epsilon: 13.27581426442283, Train ACC: 94.94%
Client: 1, Epoch: 5, Loss: 0.4833025634288788, Epsilon: 14.848469660834416, Train ACC: 94.32%
Client: 2, Epoch: 1, Loss: 0.6451927423477173, Epsilon: 7.229711840478823, Train ACC: 62.50%
Client: 2, Epoch: 2, Loss: 0.6024797558784485, Epsilon: 9.618787125042179, Train ACC: 74.36%
Client: 2, Epoch: 3, Loss: 0.5956522822380066, Epsilon: 11.56035669629223, Train ACC: 80.92%
Client: 2, Epoch: 4, Loss: 0.5821568369865417, Epsilon: 13.27581426442283, Train ACC: 83.14%
Client: 2, Epoch: 5, Loss: 0.4995638132095337, Epsilon: 14.848469660834416, Train ACC: 83.94%
Client: 3, Epoch: 1, Loss: 0.6860606670379639, Epsilon: 7.22971184047

In [100]:
# Evaluación del ACC para cada cliente con la división de entrenamiento
for i in range(3):
    accuracy = clients[i].evaluate(X_test_tensor, y_test_tensor)
    print(f'Accuracy cliente {i+1}: {accuracy * 100:.2f}%')

Accuracy cliente 1: 94.74%
Accuracy cliente 2: 84.21%
Accuracy cliente 3: 80.70%


# Paso 6. Validación del Modelo Global


In [101]:
# Evaluación del ACC del modelo global entrenado con la división de prueba
server = Server()
server.aggregate(client_models)

# Evaluación del modelo global

global_accuracy = server.evaluate(X_test_tensor, y_test_tensor)
print(f'Global model accuracy: {global_accuracy * 100:.2f}%')

Global model accuracy: 79.82%




## ✨ Inspiración

Aunque el código no es exactamente el mismo, este proyecto se inspiró en el innovador trabajo realizado por [Yang, Y., Hui, B., Yuan, H., Gong, N., & Cao, Y. (2023). PrivateFL: Accurate, Differentially Private Federated Learning via Personalized Data Transformation. In Proceedings of the USENIX Security Symposium (Usenix'23)](https://github.com/BHui97/PrivateFL), el cual ofreció valiosas percepciones sobre el aprendizaje federado y la privacidad diferencial. Queremos expresar nuestro agradecimiento a los autores originales y contribuyentes del proyecto que nos inspiró por compartir su conocimiento y recursos con la comunidad de código abierto.

